In [ ]:
# pip install torch

In [1]:
import numpy as np
import torch
import torch.nn as nn

In [36]:
# Prepare dataset
# PATH_TO_DATA = "/Users/clarec/Desktop/COMP30024-AI-ProjectB/agent/log"
state_dataset = torch.from_numpy(np.loadtxt('log/mcts_random_state.csv', delimiter=',', dtype=float).reshape(-1, 1, 98)).to(torch.float32)
action_dataset = torch.from_numpy(np.loadtxt('log/mcts_random_action.csv', delimiter=',', dtype=float).reshape(-1, 1, 4)).to(torch.float32)
reward_dataset = torch.from_numpy(np.loadtxt('log/mcts_random_reward.csv', delimiter=',', dtype=float).reshape(-1, 1, 1)).to(torch.float32)

In [37]:
print(state_dataset.shape)
print(action_dataset.shape)
print(reward_dataset.shape)

torch.Size([343, 1, 98])
torch.Size([343, 1, 4])
torch.Size([343, 1, 1])


In [45]:

class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, action_dim, reward_dim, embedding_dim, nhead, num_layers):
        super(DecisionTransformer, self).__init__() 
        self.state_embedding = nn.Linear(state_dim, embedding_dim) 
        self.action_embedding = nn.Linear(action_dim, embedding_dim) 
        self.reward_embedding = nn.Linear(reward_dim, embedding_dim)
        
        self.transformer = nn.Transformer(embedding_dim, nhead, num_layers) 
        self.output_layer = nn.Linear(embedding_dim, action_dim)
        
    def forward(self, states, actions, rewards, mask=None): 
        state_embeds = self.state_embedding(states) 
        action_embeds = self.action_embedding(actions) 
        reward_embeds = self.reward_embedding(rewards)
        input_embeds = torch.cat((state_embeds, action_embeds, reward_embeds), dim=1) 
        # tgt = torch.rand((20, 32, 512))
        # TODO: what is tgt?
        transformer_output = self.transformer(input_embeds.transpose(0, 1), src_key_padding_mask=mask, tgt=input_embeds.transpose(0, 1)) 
        action_logits = self.output_layer(transformer_output[-1])
        return action_logits
    
# state_dim = 10 
# action_dim = 5 
state_dim = 98
action_dim = 4
reward_dim = 1 
embedding_dim = 256
nhead = 4 
num_layers = 2
# n = state_dataset.shape[0]
n = 343

model = DecisionTransformer(state_dim, action_dim, reward_dim, embedding_dim, nhead, num_layers)

# Load dataset
# # states = torch.randn(32, 10, state_dim)
states = torch.randn(n, 1, state_dim)
# actions = torch.randint(0, action_dim, (32, 10, 1))
actions = torch.randn(n, 1, action_dim)
rewards = torch.randn(n, 1, reward_dim)

# states = state_dataset
# actions = action_dataset
# rewards = reward_dataset

# print(states.shape)
# print(actions.shape)
# print(rewards.shape)
# print(states.dtype)

# # Transfer to one-hot
# actions_one_hot = torch.zeros(32, 10, action_dim) 
# actions_one_hot.scatter_(2, actions, 1)

# # Got outputs of the model
# action_logits = model(states, actions_one_hot, rewards)
action_logits = model(states, actions, rewards)
# print(action_logits.shape)
# print(action_logits)
# print(actions.squeeze(-1))

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training (one batch)
optimizer.zero_grad()
# loss = loss_fn(action_logits, actions.squeeze(-1)[:, -1]) 
loss = loss_fn(torch.flatten(action_logits), torch.flatten(actions)) 
loss.backward()
optimizer.step()

torch.Size([343, 4])


In [ ]:
action_logits